1、准备工作

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import matplotlib.pyplot as plt
import seaborn

from torch.autograd import variable

seaborn.set_context(context="talk")


2、模型结构

In [ ]:
class EncoderDecoder(nn.Module):
    """一个标准的编码-解码架构"""
    #定义一个构造器
    def _init_(self,encoder,decoder,src_embd,tgt_embd,generator):
        super(EncoderDecoder,self)._init_()
        self.encoder=encoder
        self.decoder=decoder
        self.src_embd=src_embd
        self.tgt_emdb=tgt_embd
        self.generator=generator

    # take in and process masked src and target sequences
    def forward(self,src,tgt,src_mask,tgt_mask):
        return self.decode(self.encode(src,src_mask),src_mask,tgt,tgt_mask)

    def encode(self,src,src_mask):
        return self.encoder(self.src_embd(src),src_mask)

    def decode(self,memory,src_mask,tgt,tgt_mask):
        return self.decoder(self.tgt_emdb(tgt),memory,src_mask,tgt_mask)
   


In [ ]:
class Generator(nn.Module):
    "define standard linear + sofmax generation step"

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

Encoder
* 由N=6个相同的层组成

In [ ]:
def clones(module,N):
    "produce N identical layers"
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
    def _init_(self,layer,N):
        super(Encoder,self).__init__()
        self.layers=clones(layer,N) # 将一个block复制N次
        self.norm=LayerNorm(layer.size)  #进行层连接

    def forward(self,x,mask):
        "Pass the input (and mask) through each layerin turn"
        for layer in self.layers:
            x=layer(x,mask)
        return self.norm(x)

在每个子层之间使用残差连接（Residual Connection）和归一化

In [ ]:
class LayerNorm(nn.Module):
    "construct a layernorm module(See citation for deails)"
    def _init_(self,features ,eps=1e-6):
        super(LayerNorm,self).__init__()
        # torch.ones 返回一个全为1的张量 torch.zeros 返回一个全为标量0的张量
        # nn.Parameter 作为nn.Module中可训练的参数使用， 会自动被认为是module的可训练参数，nn.Parameter的对象的requires_grad属性的默认值是True
        self.a_2=nn.Parameter(torch.ones(features))
        self.b_2=nn.Parameter(torch.zeros(features))
        self.eps=eps

    def forward(self,x):
        mean=x.mean(-1,keepdim=True) # 求均值
        std=x.std(-1,keepdim=True) # 计算标准差
        return self.a_2*(x-mean)/(std + self.eps)+self.b_2


每个子层的输出为[公式]，其中[公式]是由子层自动实现的函数。我们在每个子层的输出上使用Dropout，然后将其添加到下一子层的输入并进行归一化

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self,size,dropout):
        super(SublayerConnection,self)._init_()
        self.norm=LayerNorm(size)
        self.dropout=nn.Dropout(dropout)

    def forward(self,x,sublayer):
       "Apply residual connection to any sublayer with the same size."
       return x+self.dropout(sublayer(self.norm(x)))

每层都有两个子层组成。第一个子层实现了“多头”的 Self-attention，第二个子层则是一个简单的Position-wise的全连接前馈网络

In [ ]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def _init_(self,size,self_attn,)